## 저장한 사진들과 비교

In [13]:
import cv2
import numpy as np
import os, glob

In [14]:
base_dir = './faces'
min_accuracy = 85

# LBP 
face_classifier = cv2.CascadeClassifier(base_dir+'/haarcascade_frontalface_default.xml')
model =  cv2.face.LBPHFaceRecognizer_create()
model.read(os.path.join(base_dir,'all_face.xml'))

In [15]:
# 디렉토리 이름으로 사용자 이름과 아이디 매핑정보 생성
dirs = [d for d in glob.glob(base_dir+'/*') if os.path.isdir(d)]
names = dict([])
for dir in dirs:
    dir = os.path.basename(dir)
    name,id =  dir.split('_')
    names[int(id)] = name       

In [16]:
# 카메라 준비
cap =  cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame =  cap.read()
    if not ret:
        print('no frame')
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)
    # 얼굴 검출
    faces =  face_classifier.detectMultiScale(gray, 1.3,5)
    for(x,y,w,h) in faces:
        # 얼굴영역표시 샘플과 같은 크기로 축소
        cv2.rectangle(frame, (x,y),(x+w, y+h), (0,255,255),2)
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (200,200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        # LBP 얼굴인식기로 예측
        label,confidence =  model.predict(face)
        if confidence < 400:
            # 정확도 거리를 퍼센트로 변환
            accuracy = int(100*(1-confidence/400))
            if accuracy>= min_accuracy:
                msg = '%s(%.0f%%)'%(names[label],accuracy)
            else:
                msg='Unknown'
        #사용자 이름과 정화도 결과 출력
        txt,base =  cv2.getTextSize(msg, cv2.FONT_HERSHEY_COMPLEX,1,3)
        cv2.rectangle(frame, (x,y-base-txt[1]), (x+txt[0],y+txt[1]),(0,255,255),-1 )
        cv2.putText(frame,msg,(x,y), cv2.FONT_HERSHEY_COMPLEX,1,(200,200,200),2,cv2.LINE_AA)
    cv2.imshow('Face Reconition',frame)
    if cv2.waitKey(1) == 27: # esc
        break;
cap.release()
cv2.destroyAllWindows()
    

error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\cascadedetect.cpp:1689: error: (-215:Assertion failed) !empty() in function 'cv::CascadeClassifier::detectMultiScale'
